In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 1 #当日は0、前日は1・・・
date2 = "08/17"
pmc = "10019006" # 店舗ID
SPREADSHEET_KEY = '1JRLlGQLARodLn5enVVRQwZlb6f6cL4p_CKPC7LEoJNs' # スプレッドシートのID
pstart = 20 #1 パチンコの1台目
sstart = 1001 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+123@gmail.com"
user_pw = "paruparu21"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "549":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "549":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})


['08/17', '025677', '20', '15128.0', '10834.0', '17', '4']
['08/17', '025677', '21', '12821.0', '9631.0', '14', '2']
['08/17', '025677', '22', '12821.0', '-10231.0', '1', '1']
['08/17', '025677', '23', '5128.0', '21668.0', '18', '2']
['08/17', '025677', '24', '10000.0', '-1504.0', '7', '1']
['08/17', '025677', '25', '10256.0', '-2707.0', '5', '1']
['08/17', '025677', '26', '12821.0', '-10231.0', '1', '1']
['08/17', '025677', '27', '8718.0', '7524.0', '11', '3']
['08/17', '025677', '28', '11282.0', '13242.0', '15', '4']
['08/17', '025677', '29', '9231.0', '-6920.0', '1', '1']
['08/17', '025824', '30', '0', '0', '0', '0']
['08/17', '025824', '31', '0', '0', '0', '0']
['08/17', '025824', '32', '2564.0', '-2406.0', '0', '0']
['08/17', '025873', '33', '4359.0', '2408.0', '5', '2']
['08/17', '025873', '34', '6154.0', '-5416.0', '0', '0']
['08/17', '025873', '35', '2564.0', '13844.0', '11', '1']
['08/17', '025779', '36', '9231.0', '5418.0', '17', '3']
['08/17', '025779', '37', '0', '0', '0', 

['08/17', '025895', '168', '34103.0', '-29791.0', '2', '2']
['08/17', '025895', '169', '23590.0', '-10231.0', '9', '3']
['08/17', '025895', '170', '39744.0', '-28286.0', '7', '5']
['08/17', '025895', '171', '39487.0', '-13541.0', '17', '6']
['08/17', '025665', '172', '31795.0', '-26180.0', '2', '2']
['08/17', '025665', '173', '21282.0', '-18656.0', '0', '0']
['08/17', '025665', '174', '15128.0', '24978.0', '25', '1']
['08/17', '025665', '175', '43077.0', '-21967.0', '11', '3']
['08/17', '025665', '176', '24615.0', '18959.0', '27', '3']
['08/17', '025665', '177', '9744.0', '3311.0', '9', '3']
['08/17', '025665', '178', '19487.0', '3612.0', '14', '2']
['08/17', '025665', '179', '30000.0', '22871.0', '31', '2']
['08/17', '025665', '180', '21795.0', '-1805.0', '11', '1']
['08/17', '025665', '183', '24872.0', '42431.0', '40', '4']
['08/17', '025665', '184', '22821.0', '-1203.0', '14', '3']
['08/17', '025665', '185', '5385.0', '-1504.0', '3', '1']
['08/17', '025665', '186', '35128.0', '-4212

['08/17', '025553', '325', '769.0', '-902.0', '0', '0']
['08/17', '025553', '326', '513.0', '-1203.0', '0', '0']
['08/17', '025436', '327', '0', '0', '0', '0']
['08/17', '025863', '328', '6154.0', '5117.0', '17', '4']
['08/17', '025817', '329', '6923.0', '-1203.0', '11', '3']
['08/17', '025817', '330', '10513.0', '-5115.0', '9', '2']
['08/17', '025892', '331', '7692.0', '-3911.0', '5', '2']
['08/17', '025851', '332', '513.0', '-902.0', '0', '0']
['08/17', '025821', '333', '2051.0', '-1504.0', '2', '2']
['08/17', '025285', '334', '513.0', '-902.0', '0', '0']
['08/17', '025409', '335', '513.0', '-1203.0', '0', '0']
['08/17', '025774', '336', '7436.0', '-5115.0', '3', '3']
['08/17', '025868', '337', '1282.0', '302.0', '4', '2']
['08/17', '025544', '338', '2051.0', '2709.0', '15', '1']
['08/17', '025698', '339', '2564.0', '-902.0', '5', '2']
['08/17', '025722', '340', '0', '0', '0', '0']
['08/17', '025835', '341', '17179.0', '-10832.0', '22', '8']
['08/17', '025597', '342', '4615.0', '-180

['08/17', '025605', '468', '13333.0', '-1203.0', '14', '7']
['08/17', '025877', '469', '24103.0', '3913.0', '30', '8']
['08/17', '025489', '470', '10000.0', '1205.0', '5', '2']
['08/17', '025765', '471', '19231.0', '4515.0', '29', '10']
['08/17', '025817', '472', '17949.0', '-6018.0', '27', '10']
['08/17', '025553', '473', '26154.0', '302.0', '19', '8']
['08/17', '025521', '474', '15897.0', '-6920.0', '8', '6']
['08/17', '025877', '475', '44103.0', '2408.0', '49', '17']
['08/17', '025436', '476', '8205.0', '-4212.0', '2', '1']
['08/17', '025579', '477', '26923.0', '3311.0', '14', '6']
['08/17', '025636', '478', '31026.0', '-8726.0', '17', '6']
['08/17', '025877', '479', '33077.0', '1205.0', '40', '16']
['08/17', '025521', '480', '33077.0', '-5115.0', '27', '10']
['08/17', '025579', '481', '6923.0', '-902.0', '4', '1']
['08/17', '025605', '482', '27179.0', '-8425.0', '19', '8']
['08/17', '025553', '483', '21026.0', '-4814.0', '13', '3']
['08/17', '025737', '484', '22308.0', '-11434.0', 

['08/17', '025667', '607', '4359.0', '-3309.0', '1', '1']
['08/17', '025827', '608', '17179.0', '-9027.0', '6', '3']
['08/17', '025789', '609', '21795.0', '30996.0', '33', '4']
['08/17', '025667', '610', '15128.0', '-8726.0', '3', '1']
['08/17', '025825', '611', '21538.0', '-13541.0', '4', '1']
['08/17', '025826', '612', '22821.0', '9330.0', '19', '3']
['08/17', '025868', '613', '6410.0', '-4513.0', '2', '2']
['08/17', '025764', '614', '22051.0', '-9629.0', '6', '4']
['08/17', '025275', '615', '8462.0', '-902.0', '5', '3']
['08/17', '025856', '616', '29231.0', '-2406.0', '16', '4']
['08/17', '025680', '617', '6154.0', '1506.0', '4', '1']
['08/17', '025670', '618', '20000.0', '904.0', '26', '2']
['08/17', '025562', '619', '8718.0', '-4513.0', '4', '1']
['08/17', '025606', '620', '23846.0', '-18656.0', '2', '2']
['08/17', '119916', '1001', '1383', '-602.0', '2', '5', '--']
['08/17', '119916', '1002', '3507', '370.0', '14', '11', '--']
['08/17', '119916', '1003', '3136', '1435.0', '18', '

['08/17', '120033', '1120', '3852', '-2269.0', '6', '11', '--']
['08/17', '120033', '1121', '6023', '3287.0', '29', '15', '--']
['08/17', '120137', '1122', '1800', '-417.0', '4', '3', '7']
['08/17', '120137', '1123', '2268', '-1389.0', '3', '4', '7']
['08/17', '120088', '1124', '586', '-370.0', '1', '1', '1']
['08/17', '120088', '1125', '4887', '1389.0', '17', '13', '25']
['08/17', '119945', '1126', '0', '0', '0', '0', '--']
['08/17', '119945', '1127', '5484', '-1296.0', '12', '11', '--']
['08/17', '120071', '1128', '2992', '-1389.0', '7', '10', '--']
['08/17', '120071', '1129', '1695', '-926.0', '4', '4', '--']
['08/17', '120130', '1130', '7052', '-4907.0', '0', '13', '24']
['08/17', '120130', '1131', '5457', '1991.0', '0', '7', '22']
['08/17', '120130', '1132', '2885', '-1065.0', '0', '7', '13']
['08/17', '120130', '1133', '3350', '324.0', '0', '5', '15']
['08/17', '120130', '1134', '5366', '1111.0', '0', '11', '26']
['08/17', '120142', '1135', '2238', '-2222.0', '0', '8', '8']
['08/

['08/17', '120129', '1253', '4172', '-1806.0', '0', '19', '16']
['08/17', '120108', '1254', '1071', '139.0', '0', '0', '7']
['08/17', '120108', '1255', '587', '-93.0', '0', '0', '3']
['08/17', '120108', '1256', '3284', '-2037.0', '0', '0', '17']
['08/17', '120108', '1257', '1932', '-1944.0', '0', '0', '5']
['08/17', '120104', '1258', '313', '-231.0', '0', '0', '1']
['08/17', '120104', '1259', '233', '-93.0', '0', '0', '1']
['08/17', '120104', '1260', '497', '-556.0', '0', '0', '1']
['08/17', '120104', '1261', '290', '-370.0', '0', '0', '--']
['08/17', '120104', '1262', '482', '-648.0', '0', '0', '--']
['08/17', '120104', '1263', '1435', '-324.0', '0', '2', '6']
['08/17', '120093', '1264', '5758', '1620.0', '49', '0', '49']
['08/17', '120127', '1265', '2227', '2824.0', '0', '1', '6']
['08/17', '120100', '1266', '2574', '694.0', '0', '4', '56']
['08/17', '120070', '1267', '5284', '880.0', '0', '13', '66']
['08/17', '120083', '1268', '1700', '-417.0', '0', '11', '--']
['08/17', '119987', 

['08/17', '120110', '1385', '1041', '556.0', '9', '3', '--']
['08/17', '120110', '1386', '1081', '-509.0', '5', '1', '--']
['08/17', '120110', '1387', '215', '-324.0', '0', '0', '--']
['08/17', '120110', '1388', '2393', '5231.0', '38', '9', '--']
['08/17', '120110', '1389', '2148', '-2037.0', '7', '4', '--']
['08/17', '120134', '1390', '1037', '231.0', '5', '5', '--']
['08/17', '120134', '1391', '538', '-370.0', '0', '3', '--']
['08/17', '120106', '1392', '378', '-324.0', '1', '0', '--']
['08/17', '120106', '1393', '1937', '139.0', '10', '9', '--']
['08/17', '120025', '1394', '1831', '370.0', '8', '3', '--']
['08/17', '120025', '1395', '373', '-93.0', '1', '1', '--']
['08/17', '120025', '1396', '831', '-185.0', '2', '3', '--']
['08/17', '120025', '1397', '1044', '-93.0', '4', '2', '--']
['08/17', '120127', '1398', '2928', '1481.0', '0', '1', '7']
['08/17', '120127', '1399', '2024', '-1296.0', '0', '0', '3']
['08/17', '120127', '1400', '1782', '-139.0', '0', '0', '5']
['08/17', '120127'

{'spreadsheetId': '1JRLlGQLARodLn5enVVRQwZlb6f6cL4p_CKPC7LEoJNs',
 'updates': {'spreadsheetId': '1JRLlGQLARodLn5enVVRQwZlb6f6cL4p_CKPC7LEoJNs',
  'updatedRange': 'S5RAW!F18693:M18771',
  'updatedRows': 79,
  'updatedColumns': 8,
  'updatedCells': 632}}